In [ ]:
""" Описание столбцов:
Index — номер строки
Suburb — наименование пригорода
Address — адрес
Rooms — количество комнат в помещении
Type — тип здания (h — дом, коттедж, вилла, терраса; u — блочный, дуплексный дом; t — таунхаус)
Price — цена помещения
Method — метод продажи 
SellerG — риэлторская компания
Date — дата продажи (в формате день/месяц/год)
Distance — расстояния до объекта от центра Мельбурна 
Postcode — почтовый индекс
Bedroom — количество спален
Bathroom — количество ванных комнат
Car — количество парковочных мест
Landsize — площадь прилегающей территории
BuildingArea — площадь здания
YearBuilt — год постройки
CouncilArea — региональное управление
Lattitude — географическая широта
Longitude — географическая долгота
Regionname — наименование района Мельбурна
Propertycount — количество объектов недвижимости в районе
Coordinates — широта и долгота, объединённые в кортеж
"""


In [10]:
import pandas as pd

melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')
melb_df = melb_data.copy()

melb_df = melb_df.drop(['index', 'Coordinates'], axis=1) # порядковые номера или имена столбцов для удаления, если их несколько, передаётся список
# axis=0 — удаляются строки, axis=1 — удаляются столбцы

total_rooms = melb_df['Rooms'] + melb_df['Bedroom'] + melb_df['Bathroom'] # общее количество комнат
melb_df['MeanRoomsSquare'] = melb_df['BuildingArea'] / total_rooms # средняя площадь одной комнаты

diff_area = melb_df['BuildingArea'] - melb_df['Landsize']
sum_area = melb_df['BuildingArea'] + melb_df['Landsize']
melb_df['AreaRatio'] = diff_area/sum_area # соотношение площади здания и участка
# <0 площадь участка больше площади здания, -1 без строений, 1 без территорий